<a href="https://colab.research.google.com/github/seripoyo/wp-prompt/blob/main/%E5%A4%96%E5%9B%BD%E4%BA%BA%E9%9B%87%E7%94%A8%E5%85%A5%E7%A8%BF%E3%83%97%E3%83%AD%E3%83%B3%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリをインストールします
!pip install markdown -q

import re
import markdown
import json
import os
from google.colab import files
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 変換ロジックの関数
# -------------------------------------------------------------------
def convert_specific_md_to_html(md_text):
    """
    特定のルールに基づいてマークダウンテキストをHTMLに変換する関数。
    """
    processed_text = md_text

    # --- ステップ1: ブロックレベルのカスタム構文を先にHTMLに置換 ---
    def replace_freebox(match):
        title = match.group(1).strip()
        content = match.group(2).strip()
        content_html = markdown.markdown(content, extensions=['tables'])
        return (f'<div style="padding: 10px 0;">\n'
                f'<div class="freebox"><p class="p-free"><span class="p-entry-f">{title}</span></p><div class="free-inbox">{content_html}</div></div></div>')
    processed_text = re.sub(r'^★\*\*(.*?)\*\*\s*([\s\S]*?)\s*★$', replace_freebox, processed_text, flags=re.MULTILINE)

    # ★★★ [ ] や ■ ■ で囲まれたブロックをテーブルに変換するロジック ★★★
    def replace_block_with_styled_table(match):
        content = match.group(1).strip()

        # コンテンツを段落（空行区切り）で分割し、空の要素は除去
        paragraphs = [p.strip() for p in re.split(r'\n\s*\n', content) if p.strip()]

        if not paragraphs:
            return ""

        processed_paragraphs = []
        for para in paragraphs:
            # **...** でテキストを分割する。キャプチャグループ `()` を使うことで、
            # デリミタ自体も結果のリストに含まれるようにする。
            parts = re.split(r'(\*\*.*?\*\*)', para)

            processed_para_parts = []
            for part in parts:
                # 空の文字列はスキップ
                if not part:
                    continue

                # テキスト内の改行を <br /> に変換
                part_with_br = part.replace('\n', '<br />\n')

                # partが **...** の形式かチェック
                if part.startswith('**') and part.endswith('**'):
                    # ** を取り除いた部分のテキストを取得
                    inner_text_with_br = part_with_br[2:-2]
                    processed_para_parts.append(f'<strong>{inner_text_with_br}</strong>')
                else:
                    # 太字でない部分はspanで囲む
                    processed_para_parts.append(f'<span style="font-weight: 400;">{part_with_br}</span>')

            # 1つの段落を構成するHTMLパーツを結合
            processed_paragraphs.append(''.join(processed_para_parts))

        # 処理済みの各段落を改行文字で結合
        content_html = '\n'.join(processed_paragraphs)

        return (f'<table style="border-collapse: collapse; width: 100%;">\n'
                f'<tbody>\n'
                f'<tr><td style="width: 100%;">\n{content_html}\n</td>\n'
                f'</tr>\n'
                f'</tbody>\n'
                f'</table>')

    # [ ... ] のブロックを処理
    processed_text = re.sub(r'^\[\s*?$([\s\S]*?)^\]\s*?$', replace_block_with_styled_table, processed_text, flags=re.MULTILINE)

    # ■ ... ■ のブロックを処理（同じ関数を再利用）
    processed_text = re.sub(r'^■\s*?$([\s\S]*?)^■\s*?$', replace_block_with_styled_table, processed_text, flags=re.MULTILINE)


    # --- ステップ2: インラインのカスタム構文や単純なショートコード置換 ---
    processed_text = processed_text.replace("【目次】", "[toc]")
    processed_text = processed_text.replace("【山根さん監修】", "[sc name=\"yamane_kansyu\"][/sc]")
    processed_text = processed_text.replace("【特定技能外国人の受入れ診断チェック】", "[sc name=\"sindancheck\"][/sc]")
    processed_text = processed_text.replace("【最終CTA】", "[sc name=\"last_cta\"][/sc]")

    # --- ステップ3: リンクにtarget="_blank"を追加するための前処理 ---
    processed_text = re.sub(r'^(参考：\s*)(\[.*?\]\(.*?\))', r'\1\2{:target="_blank" rel="noopener"}', processed_text, flags=re.MULTILINE)

    # --- ステップ4: 標準MarkdownからHTMLへの変換 ---
    html = markdown.markdown(processed_text, extensions=['tables', 'attr_list'])

    # --- ステップ5: HTMLに対する後処理 ---
    def style_table_header(match):
        thead_content = match.group(1)
        styled_thead_content = re.sub(r'<th>', '<th style="background-color: #f0f0f0; font-weight: bold;">', thead_content)
        return f'<thead>{styled_thead_content}</thead>'
    html = re.sub(r'<thead>([\s\S]*?)</thead>', style_table_header, html)

    h2_count = 0
    def replace_h2(match):
        nonlocal h2_count
        h2_count += 1
        content = match.group(1)
        tag_id = "i" if h2_count == 1 else f"i-{h2_count}"
        return f'<h2><span id="{tag_id}"></span>{content}</h2>'

    h3_count = 0
    def replace_h3(match):
        nonlocal h3_count
        h3_count += 1
        content = match.group(1)
        tag_id = "i-2" if h3_count == 1 else f"i-2-{h3_count}"
        return f'<h3><span id="{tag_id}"></span>{content}</h3>'

    html = re.sub(r'<h2>(.*?)</h2>', replace_h2, html)
    html = re.sub(r'<h3>(.*?)</h3>', replace_h3, html)

    def add_target_to_related_links(match):
        section_html = match.group(0)
        return re.sub(r'<a href="', '<a target="_blank" rel="noopener" href="', section_html)
    html = re.sub(r'(<h[1-6]>【関連記事】</h[1-6]>\s*<ul>[\s\S]*?</ul>)', add_target_to_related_links, html)

    return html

# -------------------------------------------------------------------
# メインの実行部分
# -------------------------------------------------------------------
def main():
    """アップロードされたMDファイルを探して変換処理を実行する"""
    markdown_file_name = None
    search_directory = '/content/'

    for filename in os.listdir(search_directory):
        if filename.endswith('.md'):
            markdown_file_name = filename
            break

    if markdown_file_name:
        print(f"✅ マークダウンファイル '{markdown_file_name}' を検出しました。変換処理を開始します...")
        file_path = os.path.join(search_directory, markdown_file_name)

        with open(file_path, 'r', encoding='utf-8') as f:
            md_content = f.read()

        html_output = convert_specific_md_to_html(md_content)

        # --- 結果の表示とコピーボタンの実装 ---
        escaped_html = json.dumps(html_output)
        display_code = f"""
        <h4>変換後のHTML</h4>
        <textarea id="html-output" style="width: 95%; height: 250px; font-family: monospace; border: 1px solid #ccc; padding: 5px;" readonly>{html_output}</textarea>
        <br>
        <button onclick="copyToClipboard()" style="padding: 8px 15px; border-radius: 5px; background-color: #4CAF50; color: white; border: none; cursor: pointer;">
          クリップボードにコピー
        </button>
        <span id="copy-status" style="margin-left: 10px; color: green; font-weight: bold;"></span>

        <script>
        function copyToClipboard() {{
            const textToCopy = {escaped_html};
            navigator.clipboard.writeText(textToCopy).then(() => {{
                const status = document.getElementById('copy-status');
                status.textContent = 'コピーしました！';
                setTimeout(() => {{ status.textContent = ''; }}, 2000);
            }}).catch(err => {{
                console.error('コピーに失敗しました: ', err);
                const status = document.getElementById('copy-status');
                status.textContent = 'コピーに失敗しました。';
                setTimeout(() => {{ status.textContent = ''; }}, 2000);
            }});
        }}
        </script>
        """
        display(HTML(display_code))

    else:
        print("⚠️ マークダウンファイルが見つかりませんでした。")
        print("左側のファイルパネルに .md ファイルをアップロードしてから、このセルを再度実行してください。")

# スクリプトを実行
main()

✅ マークダウンファイル '外国人のプロンプト用原稿サンプル 02.md' を検出しました。変換処理を開始します...
